## Login with az cli

use az cli to login from the shell or powershell
```shell
az logout
az login --tenant "<tenent_id>" --scope "https://management.azure.com/.default" --scope "https://ai.azure.com/.default"
```

make sure azure cli login user is either "owner" or "contributor" in the hub based project

references:
* https://learn.microsoft.com/en-us/azure/ai-foundry/how-to/fine-tune-serverless?tabs=chat-completion&pivots=programming-language-python

In [8]:
# azure machine learnin client
from azure.ai.ml import MLClient
from azure.ai.ml.entities import Model
from dotenv import load_dotenv

load_dotenv(dotenv_path=".env", override=True)

from utils.hubauth import AuthHelper
settings = AuthHelper.load_settings()
credential = AuthHelper.test_credential()

if credential:
    print('Environment and authentication OK')
else:
    print("please login first")

Environment and authentication OK


In [9]:
try:
    workspace_ml_client = MLClient.from_config(credential=credential)
except:
    workspace_ml_client = MLClient(
        credential=credential,
        subscription_id=settings.subscription_id,
        resource_group_name=settings.resource_group_name,
        workspace_name=settings.project_name,
    )

# the models, fine tuning pipelines and environments are available in various AzureML system registries,
# Example: Phi family of models are in "azureml", Llama family of models are in "azureml-meta" registry.
registry_ml_client = MLClient(credential, registry_name="azureml")

# Get AzureML workspace object, the workspace_name must be a hub based workspace name
workspace = workspace_ml_client._workspaces.get(workspace_ml_client.workspace_name)
workspace.id

Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented


'/subscriptions/6753a2ee-12b7-4fac-82fa-48824fb58abe/resourceGroups/rg-ai-sandbox-yw-uno/providers/Microsoft.MachineLearningServices/workspaces/ai-proj-demo-yw-uno'

In [3]:
# workspaces =workspace_ml_client._workspaces.list()
# for ws in workspaces:
#     print(f"Workspace: {ws.name}, ID: {ws.id}, Location: {ws.location}")

In [7]:
# model with fine-tuning support
model_name = "Phi-4-mini-instruct"
model_to_finetune = registry_ml_client.models.get(model_name, label="latest")
print(
    "\n\nUsing model name: {0}, version: {1}, id: {2} for fine tuning".format(
        model_to_finetune.name, model_to_finetune.version, model_to_finetune.id
    )
)



Using model name: Phi-4-mini-instruct, version: 1, id: azureml://registries/azureml/models/Phi-4-mini-instruct/versions/1 for fine tuning


In [16]:
import os
parent_dir =os.getcwd()
data_dir = os.path.join(parent_dir, "data")
model_dir = os.path.join(parent_dir, "models")
train_data_path = os.path.join(data_dir, "train.jsonl")
validation_data_path = os.path.join(data_dir, "validation.jsonl")

print(train_data_path)
print(model_dir)

c:\Users\yingdingwang\Documents\VCS\democollections\model-fine-tuning\01-serverless\hub\data\train.jsonl
c:\Users\yingdingwang\Documents\VCS\democollections\model-fine-tuning\01-serverless\hub\models


In [ ]:
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml.entities import Data
# make sure the storage account aihubdemoywuno has public network access, and your az login user has Blob Storage Contributor role
# minimum 65 rows in train.jsonl
dataset_version = "2"
train_dataset_name = "chat_training_small"
try:
    train_data_asset = workspace_ml_client.data.get(
        name=train_dataset_name, version=dataset_version
    )
    print(f"Dataset {train_dataset_name} already exists")
except:
    print("creating dataset")
    train_data = Data(
        path=train_data_path,
        type=AssetTypes.URI_FILE,
        description="Training dataset",
        name=train_dataset_name,
        version=dataset_version,
    )
    train_data_asset = workspace_ml_client.data.create_or_update(train_data)

creating dataset


In [ ]:
from azure.ai.ml.entities import Data

dataset_version = "1"
validation_dataset_name = "chat_validation_small"
try:
    validation_data_asset = workspace_ml_client.data.get(
        validation_dataset_name, version=dataset_version
    )
    print(f"Dataset {validation_dataset_name} already exists")
except:
    print("creating dataset")
    validation_data = Data(
        path=validation_data_path,
        type=AssetTypes.URI_FILE,
        description="Validation dataset",
        name=validation_dataset_name,
        version="1",
    )
    validation_data_asset = workspace_ml_client.data.create_or_update(validation_data)

creating dataset


Uploading validation.jsonl (< 1 MB): 100%|██████████| 1.80k/1.80k [00:00<00:00, 14.8kB/s]




In [ ]:
# model_id_to_subscribe = "/".join(model_to_finetune.id.split("/")[:-2])
# print(model_id_to_subscribe)

# normalized_model_name = model_name.replace(".", "-")

In [ ]:
# from azure.ai.ml.entities import MarketplaceSubscription

# subscription_name = f"{normalized_model_name}-sub"

# marketplace_subscription = MarketplaceSubscription(
#     model_id=model_id_to_subscribe,
#     name=subscription_name,
# )

# note: this will throw exception if the subscription already exists or subscription is not required (for example, if the model is not in the marketplace like Phi family)
# try:
#     marketplace_subscription = (
#         workspace_ml_client.marketplace_subscriptions.begin_create_or_update(
#             marketplace_subscription
#         ).result()
#     )
# except Exception as ex:
#     print(ex)

In [17]:
## submit fine tuning job

from azure.ai.ml.finetuning import FineTuningTaskType, create_finetuning_job
import uuid

guid = uuid.uuid4()
short_guid = str(guid)[:8]
display_name = f"{model_name}-display-name-{short_guid}-from-sdk"
name = f"{model_name}t-{short_guid}-from-sdk"
output_model_name_prefix = f"{model_name}-{short_guid}-from-sdk-finetuned"
experiment_name = f"{model_name}-from-sdk"

finetuning_job = create_finetuning_job(
    task=FineTuningTaskType.CHAT_COMPLETION,
    training_data=train_data_asset.id,
    validation_data=validation_data_asset.id,
    hyperparameters={
        "per_device_train_batch_size": "1",
        "learning_rate": "0.00002",
        "num_train_epochs": "1",
    },
    model=model_to_finetune.id,
    display_name=display_name,
    name=name,
    experiment_name=experiment_name,
    tags={"foo_tag": "bar"},
    properties={"my_property": "my_value"},
    output_model_name_prefix=output_model_name_prefix,
)

In [18]:
created_job = workspace_ml_client.jobs.create_or_update(finetuning_job)
workspace_ml_client.jobs.get(created_job.name)

pathOnCompute is not a known attribute of class <class 'azure.ai.ml._restclient.v2023_04_01_preview.models._models_py3.MLFlowModelJobOutput'> and will be ignored


Experiment,Name,Type,Status,Details Page
Phi-4-mini-instruct-from-sdk,Phi-4-mini-instructt-b4c828af-from-sdk,finetuning,Running,Link to Azure Machine Learning studio


In [ ]:
status = workspace_ml_client.jobs.get(created_job.name).status

import time

while True:
    status = workspace_ml_client.jobs.get(created_job.name).status
    print(f"Current job status: {status}")
    if status in ["Failed", "Completed", "Canceled"]:
        print("Job has finished with status: {0}".format(status))
        break
    else:
        print("Job is still running. Checking again in 30 seconds.")
        time.sleep(30)

Current job status: Running
Job is still running. Checking again in 30 seconds.
Current job status: Running
Job is still running. Checking again in 30 seconds.
Current job status: Running
Job is still running. Checking again in 30 seconds.
Current job status: Running
Job is still running. Checking again in 30 seconds.
Current job status: Running
Job is still running. Checking again in 30 seconds.
Current job status: Running
Job is still running. Checking again in 30 seconds.
Current job status: Running
Job is still running. Checking again in 30 seconds.
Current job status: Running
Job is still running. Checking again in 30 seconds.
Current job status: Running
Job is still running. Checking again in 30 seconds.


In [ ]:
finetune_model_name = created_job.outputs["registered_model"]["name"]
finetune_model_name

### deploy fine tuned model

https://learn.microsoft.com/en-us/azure/ai-foundry/how-to/fine-tune-serverless?tabs=chat-completion&pivots=programming-language-python#deploy-a-fine-tuned-model-1

In [25]:
# deploy the fine tuned model

finetune_model_name="Phi-4-mini-instruct-b4c828af-from-sdk-finetuned"

# Deploy the model as a serverless endpoint

# endpoint_name = f"{normalized_model_name}-ft-{short_guid}"  # Name must be unique
# asset id in AML model
model_id = f"azureml://locations/{workspace.location}/workspaces/{workspace._workspace_id}/models/{finetune_model_name}/versions/1"
model_id

'azureml://locations/eastus2/workspaces/64f7576c-9119-4c04-9385-fc8e67e87b34/models/Phi-4-mini-instruct-b4c828af-from-sdk-finetuned/versions/1'

In [28]:
# promptflow deployment also listed as model in the workspace
models = workspace_ml_client.models.list(name=finetune_model_name)
for model in models:
    print(f"Model: {model.name}, Version: {model.version}, ID: {model.id}")

model_name = model.name
print(f"Model name: {model_name}")

Model: Phi-4-mini-instruct-b4c828af-from-sdk-finetuned, Version: 1, ID: /subscriptions/6753a2ee-12b7-4fac-82fa-48824fb58abe/resourceGroups/rg-ai-sandbox-yw-uno/providers/Microsoft.MachineLearningServices/workspaces/ai-proj-demo-yw-uno/models/Phi-4-mini-instruct-b4c828af-from-sdk-finetuned/versions/1
Model name: Phi-4-mini-instruct-b4c828af-from-sdk-finetuned


In [38]:
## storage account public network access "aihubdemoywuno"
print(f"finetune_model_name: {finetune_model_name}")
model_path = os.path.join(model_dir, finetune_model_name)
if model_path and not os.path.exists(model_path):
    os.makedirs(model_path)

workspace_ml_client.models.download(name=finetune_model_name, version="1", download_path=model_dir)

finetune_model_name: Phi-4-mini-instruct-b4c828af-from-sdk-finetuned


In [ ]:
import azureml-mlflow 

local_path = mlflow.azureml.download_model(model_uri=model_id, dst_path=model_dir)

AttributeError: module 'mlflow' has no attribute 'azureml'